In [1]:
import oracledb
import pandas as pd

**Establishing connection to the ODB**

In [2]:
# Connect using the appropriate credentials
username = "S25_km242"
password = "uc38wTKk"
dsn = "oracle2.wiu.edu:1521/orclpdb1"  

try:
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("✅ Successfully connected to Oracle Database")

    # Create a cursor for executing SQL
    cursor = connection.cursor()

except oracledb.DatabaseError as e:
    print("❌ Error connecting to the Oracle database:", e)

✅ Successfully connected to Oracle Database


**Creating tables**

In [9]:
# --- Step 1: Create LibraryBranches ---
cursor.execute("""
CREATE TABLE LibraryBranches (
    BranchID INT PRIMARY KEY,
    BranchName VARCHAR2(255) NOT NULL,
    Address VARCHAR2(255) NOT NULL,
    ContactNumber VARCHAR2(20) NOT NULL
)
""")

# --- Step 2: Create LibraryMembers ---
cursor.execute("""
CREATE TABLE LibraryMembers (
    MemberID INT PRIMARY KEY,
    FirstName VARCHAR2(100) NOT NULL,
    LastName VARCHAR2(100) NOT NULL,
    DateOfBirth DATE NOT NULL,
    Email VARCHAR2(255) UNIQUE NOT NULL,
    Phone VARCHAR2(20) UNIQUE NOT NULL,
    Address VARCHAR2(255) NOT NULL,
    MembershipType VARCHAR2(20) CHECK (MembershipType IN ('Student', 'Faculty', 'Visitor')),
    MembershipStartDate DATE DEFAULT SYSDATE,
    OutstandingFines NUMBER(6,2) DEFAULT 0 CHECK (OutstandingFines >= 0),
    CurrentReservations INT DEFAULT 0 CHECK (CurrentReservations <= 5)
)
""")

# --- Step 3: Create Librarians ---
cursor.execute("""
CREATE TABLE Librarians (
    LibrarianID INT PRIMARY KEY,
    FirstName VARCHAR2(100) NOT NULL,
    LastName VARCHAR2(100) NOT NULL,
    Email VARCHAR2(255) UNIQUE NOT NULL,
    Phone VARCHAR2(20) UNIQUE NOT NULL,
    Role VARCHAR2(20) CHECK (Role IN ('Admin', 'Staff', 'Assistant')),
    HireDate DATE DEFAULT SYSDATE,
    Salary NUMBER(8,2) CHECK (Salary >= 0),
    BranchID INT,
    FOREIGN KEY (BranchID) REFERENCES LibraryBranches(BranchID)
)
""")

# --- Step 4: Create Books ---
cursor.execute("""
CREATE TABLE Books (
    BookID INT PRIMARY KEY,
    Title VARCHAR2(255) NOT NULL,
    ISBN VARCHAR2(20) UNIQUE NOT NULL,
    Author VARCHAR2(255) NOT NULL,
    Publisher VARCHAR2(255) NOT NULL,
    PublicationYear INT CHECK (PublicationYear >= 1500),
    Genre VARCHAR2(100),
    BookType VARCHAR2(20) CHECK (BookType IN ('Physical', 'Ebook', 'Research Paper')),
    ShelfLocation VARCHAR2(50),
    BranchID INT NOT NULL,
    TotalCopies INT CHECK (TotalCopies >= 1),
    AvailableCopies INT,
    FileFormat VARCHAR2(50),
    AccessURL VARCHAR2(255),
    FOREIGN KEY (BranchID) REFERENCES LibraryBranches(BranchID),
    CHECK (AvailableCopies >= 0 AND AvailableCopies <= TotalCopies)
)
""")

# --- Step 5: Create Loans ---
cursor.execute("""
CREATE TABLE Loans (
    LoanID INT PRIMARY KEY,
    BookID INT NOT NULL,
    MemberID INT NOT NULL,
    LoanDate DATE DEFAULT SYSDATE,
    DueDate DATE NOT NULL,
    ReturnDate DATE,
    Status VARCHAR2(20) CHECK (Status IN ('Borrowed', 'Returned', 'Overdue')),
    FineAmount NUMBER(6,2) DEFAULT 0 CHECK (FineAmount >= 0),
    ReturnCondition VARCHAR2(20) CHECK (ReturnCondition IN ('Good', 'Damaged', 'Lost')),
    DamageFee NUMBER(6,2) DEFAULT 0 CHECK (DamageFee >= 0),
    FOREIGN KEY (BookID) REFERENCES Books(BookID),
    FOREIGN KEY (MemberID) REFERENCES LibraryMembers(MemberID),
    CHECK (DueDate > LoanDate)
)
""")

# --- Step 6: Create Transactions ---
cursor.execute("""
CREATE TABLE Transactions (
    TransactionID INT PRIMARY KEY,
    MemberID INT NOT NULL,
    BookID INT NOT NULL,
    TransactionType VARCHAR2(20) 
        CHECK (TransactionType IN ('Reservation', 'Checkout', 'Return', 'Fine Payment')),
    TransactionDate DATE DEFAULT SYSDATE,
    Amount NUMBER(6,2) DEFAULT 0 CHECK (Amount >= 0),
    Status VARCHAR2(20) 
        CHECK (Status IN ('Pending', 'Completed', 'Cancelled')),
    LibrarianID INT,
    FOREIGN KEY (MemberID) REFERENCES LibraryMembers(MemberID),
    FOREIGN KEY (BookID) REFERENCES Books(BookID),
    FOREIGN KEY (LibrarianID) REFERENCES Librarians(LibrarianID)
)
""")

# Commit changes
connection.commit()
print("✅ All tables created successfully!")

✅ All tables created successfully!


**Creating VIEWS in DB**

In [10]:
#View_AvailableBooks
cursor.execute("""
CREATE OR REPLACE VIEW View_AvailableBooks AS
SELECT BookID, Title, Author, Genre, BookType, AvailableCopies, BranchID
FROM Books
WHERE AvailableCopies > 0
""")

#View_MemberLoans
cursor.execute("""
CREATE OR REPLACE VIEW View_MemberLoans AS
SELECT L.LoanID, B.Title, L.LoanDate, L.DueDate, L.ReturnDate, L.Status, L.FineAmount
FROM Loans L
JOIN Books B ON L.BookID = B.BookID
""")

#View_MemberReservations
cursor.execute("""
CREATE OR REPLACE VIEW View_MemberReservations AS
SELECT T.TransactionID, B.Title, T.TransactionDate, T.Status
FROM Transactions T
JOIN Books B ON T.BookID = B.BookID
WHERE T.TransactionType = 'Reservation'
""")

#View_MemberFines
cursor.execute("""
CREATE OR REPLACE VIEW View_MemberFines AS
SELECT T.TransactionID, B.Title, T.Amount, T.Status
FROM Transactions T
JOIN Books B ON T.BookID = B.BookID
WHERE T.TransactionType = 'Fine Payment' AND T.Status = 'Pending'
""")

#View_LibrarianBookInventory
cursor.execute("""
CREATE OR REPLACE VIEW View_LibrarianBookInventory AS
SELECT BookID, Title, ISBN, Author, Publisher, Genre, TotalCopies, AvailableCopies, BranchID
FROM Books
""")

#View_LibrarianLoans
cursor.execute("""
CREATE OR REPLACE VIEW View_LibrarianLoans AS
SELECT L.LoanID, B.Title, M.FirstName || ' ' || M.LastName AS Borrower,
       L.LoanDate, L.DueDate, L.ReturnDate, L.Status
FROM Loans L
JOIN Books B ON L.BookID = B.BookID
JOIN LibraryMembers M ON L.MemberID = M.MemberID
""")

#View_LibrarianReservations
cursor.execute("""
CREATE OR REPLACE VIEW View_LibrarianReservations AS
SELECT T.TransactionID, B.Title, M.FirstName || ' ' || M.LastName AS MemberName,
       T.TransactionDate, T.Status
FROM Transactions T
JOIN Books B ON T.BookID = B.BookID
JOIN LibraryMembers M ON T.MemberID = M.MemberID
WHERE T.TransactionType = 'Reservation'
""")

#View_LibrarianTransactions
cursor.execute("""
CREATE OR REPLACE VIEW View_LibrarianTransactions AS
SELECT T.TransactionID, M.FirstName || ' ' || M.LastName AS Member,
       B.Title, T.TransactionType, T.Amount, T.TransactionDate, T.Status
FROM Transactions T
JOIN Books B ON T.BookID = B.BookID
JOIN LibraryMembers M ON T.MemberID = M.MemberID
""")

#View_AdminLibraryStats
cursor.execute("""
CREATE OR REPLACE VIEW View_AdminLibraryStats AS
SELECT 
    (SELECT COUNT(*) FROM Books) AS TotalBooks,
    (SELECT COUNT(*) FROM Loans WHERE Status = 'Borrowed') AS ActiveLoans,
    (SELECT COUNT(*) FROM Transactions WHERE TransactionType = 'Reservation' AND Status = 'Pending') AS ActiveReservations,
    (SELECT SUM(Amount) FROM Transactions WHERE TransactionType = 'Fine Payment' AND Status = 'Pending') AS UnpaidFines
FROM dual
""")

#View_AdminUserActivity
cursor.execute("""
CREATE OR REPLACE VIEW View_AdminUserActivity AS
SELECT M.MemberID, M.FirstName || ' ' || M.LastName AS MemberName,
       COUNT(DISTINCT L.LoanID) AS TotalLoans,
       SUM(CASE WHEN T.TransactionType = 'Fine Payment' AND T.Status = 'Pending' THEN T.Amount ELSE 0 END) AS UnpaidFines
FROM LibraryMembers M
LEFT JOIN Loans L ON M.MemberID = L.MemberID
LEFT JOIN Transactions T ON M.MemberID = T.MemberID
GROUP BY M.MemberID, M.FirstName, M.LastName
""")

#View_AdminOverdueBooks
cursor.execute("""
CREATE OR REPLACE VIEW View_AdminOverdueBooks AS
SELECT L.LoanID, B.Title, M.FirstName || ' ' || M.LastName AS Borrower,
       L.DueDate, L.ReturnDate, L.FineAmount
FROM Loans L
JOIN Books B ON L.BookID = B.BookID
JOIN LibraryMembers M ON L.MemberID = M.MemberID
WHERE L.Status = 'Overdue'
""")

#View_AdminFinancialReport
cursor.execute("""
CREATE OR REPLACE VIEW View_AdminFinancialReport AS
SELECT 
    SUM(CASE WHEN Status = 'Completed' THEN Amount ELSE 0 END) AS TotalCollectedFines,
    SUM(CASE WHEN Status = 'Pending' THEN Amount ELSE 0 END) AS UnpaidFines
FROM Transactions
WHERE TransactionType = 'Fine Payment'
""")

connection.commit()
print("✅ All views created successfully!")

✅ All views created successfully!


**Creating TRIGGERS**

In [11]:
cursor.execute("""
CREATE OR REPLACE TRIGGER trg_check_age
BEFORE INSERT ON LibraryMembers
FOR EACH ROW
BEGIN
    IF :NEW.DateOfBirth > ADD_MONTHS(SYSDATE, -12*18) THEN
        RAISE_APPLICATION_ERROR(-20001, 'Member must be at least 18 years old.');
    END IF;
END;
""")

cursor.execute("""
CREATE OR REPLACE TRIGGER trg_update_available_copies
AFTER INSERT OR UPDATE ON Loans
FOR EACH ROW
BEGIN
    IF INSERTING OR (:NEW.Status = 'Borrowed' AND :OLD.Status IS NULL) THEN
        UPDATE Books SET AvailableCopies = AvailableCopies - 1
        WHERE BookID = :NEW.BookID;
    ELSIF UPDATING AND :NEW.Status = 'Returned' AND :OLD.Status = 'Borrowed' THEN
        UPDATE Books SET AvailableCopies = AvailableCopies + 1
        WHERE BookID = :NEW.BookID;
    END IF;
END;
""")

cursor.execute("""
CREATE OR REPLACE TRIGGER trg_calculate_fine
BEFORE UPDATE ON Loans
FOR EACH ROW
BEGIN
    IF :NEW.Status = 'Returned' AND :NEW.ReturnDate > :NEW.DueDate THEN
        :NEW.FineAmount := (:NEW.ReturnDate - :NEW.DueDate) * 1;
    ELSE
        :NEW.FineAmount := 0;
    END IF;
END;
""")

cursor.execute("""
CREATE OR REPLACE TRIGGER trg_set_loan_status
BEFORE UPDATE ON Loans
FOR EACH ROW
BEGIN
    IF :NEW.ReturnDate IS NOT NULL THEN
        :NEW.Status := 'Returned';
    ELSIF :NEW.DueDate < SYSDATE AND :NEW.ReturnDate IS NULL THEN
        :NEW.Status := 'Overdue';
    END IF;
END;
""")

connection.commit()
print("✅ All 4 triggers created successfully.")

✅ All 4 triggers created successfully.


**Adding sample points for testing**

In [12]:
sample_inserts = [
    # ===== LibraryBranches =====
    "INSERT INTO LibraryBranches VALUES (1, 'Central Library', '100 Main St, Metropolis', '555-1234')",
    "INSERT INTO LibraryBranches VALUES (2, 'Eastside Branch', '200 East Ave, Metropolis', '555-5678')",
    "INSERT INTO LibraryBranches VALUES (3, 'Westside Branch', '300 West Blvd, Metropolis', '555-8765')",

    # ===== LibraryMembers =====
    "INSERT INTO LibraryMembers VALUES (1001, 'James', 'Smith', TO_DATE('1985-07-15', 'YYYY-MM-DD'), 'james.smith@example.com', '555-1111', '123 Maple St, Metropolis', 'Student', SYSDATE - INTERVAL '2' YEAR, 0, 5)",
    "INSERT INTO LibraryMembers VALUES (1002, 'Maria', 'Garcia', TO_DATE('1990-05-22', 'YYYY-MM-DD'), 'maria.garcia@example.com', '555-2222', '456 Oak St, Metropolis', 'Student', SYSDATE - INTERVAL '1' YEAR, 10, 3)",
    "INSERT INTO LibraryMembers VALUES (1003, 'Robert', 'Johnson', TO_DATE('1978-11-02', 'YYYY-MM-DD'), 'robert.johnson@example.com', '555-3333', '789 Pine St, Metropolis', 'Faculty', SYSDATE - INTERVAL '3' YEAR, 5, 2)",
    "INSERT INTO LibraryMembers VALUES (1004, 'Linda', 'Martinez', TO_DATE('1982-03-14', 'YYYY-MM-DD'), 'linda.martinez@example.com', '555-4444', '101 Elm St, Metropolis', 'Faculty', SYSDATE - INTERVAL '4' MONTH, 0, 1)",
    "INSERT INTO LibraryMembers VALUES (1005, 'David', 'Brown', TO_DATE('1995-09-30', 'YYYY-MM-DD'), 'david.brown@example.com', '555-5555', '202 Birch St, Metropolis', 'Visitor', SYSDATE - INTERVAL '6' MONTH, 2, 4)",

    # ===== Librarians =====
    "INSERT INTO Librarians VALUES (2001, 'Emily', 'Davis', 'emily.davis@example.com', '555-6666', 'Admin', SYSDATE - INTERVAL '5' YEAR, 55000, 1)",
    "INSERT INTO Librarians VALUES (2002, 'Michael', 'Wilson', 'michael.wilson@example.com', '555-7777', 'Staff', SYSDATE - INTERVAL '3' YEAR, 45000, 2)",
    "INSERT INTO Librarians VALUES (2003, 'Sarah', 'Lee', 'sarah.lee@example.com', '555-8888', 'Assistant', SYSDATE - INTERVAL '1' YEAR, 40000, 3)",

    # ===== Books =====
    "INSERT INTO Books VALUES (3001, 'To Kill a Mockingbird', '9780061120084', 'Harper Lee', 'J.B. Lippincott & Co.', 1960, 'Fiction', 'Physical', 'A1', 1, 5, 5, NULL, NULL)",
    "INSERT INTO Books VALUES (3002, '1984', '9780451524935', 'George Orwell', 'Secker & Warburg', 1949, 'Dystopian', 'Physical', 'A2', 1, 4, 4, NULL, NULL)",
    "INSERT INTO Books VALUES (3003, 'Pride and Prejudice', '9781503290563', 'Jane Austen', 'T. Egerton', 1813, 'Romance', 'Physical', 'A3', 1, 3, 3, NULL, NULL)",
    "INSERT INTO Books VALUES (3004, 'The Great Gatsby', '9780743273565', 'F. Scott Fitzgerald', 'Charles Scribner''s Sons', 1925, 'Tragedy', 'Physical', 'A4', 1, 6, 6, NULL, NULL)",
    "INSERT INTO Books VALUES (3005, 'Moby-Dick', '9781503280786', 'Herman Melville', 'Harper & Brothers', 1851, 'Adventure', 'Physical', 'B1', 2, 2, 2, NULL, NULL)",
    "INSERT INTO Books VALUES (3006, 'Brave New World', '9780060850524', 'Aldous Huxley', 'Chatto & Windus', 1932, 'Science Fiction', 'Ebook', NULL, 2, 2, 2, 'PDF', 'http://library.edu/ebooks/brave_new_world.pdf')",
    "INSERT INTO Books VALUES (3007, 'The Catcher in the Rye', '9780316769488', 'J.D. Salinger', 'Little, Brown and Company', 1951, 'Coming-of-Age', 'Physical', 'B2', 2, 4, 4, NULL, NULL)",
    "INSERT INTO Books VALUES (3008, 'The Hobbit', '9780547928227', 'J.R.R. Tolkien', 'George Allen & Unwin', 1937, 'Fantasy', 'Physical', 'B3', 3, 3, 3, NULL, NULL)",
    "INSERT INTO Books VALUES (3009, 'Crime and Punishment', '9780143058144', 'Fyodor Dostoevsky', 'The Russian Messenger', 1866, 'Philosophical Fiction', 'Ebook', NULL, 3, 1, 1, 'EPUB', 'http://library.edu/ebooks/crime_and_punishment.epub')",
    "INSERT INTO Books VALUES (3010, 'Jane Eyre', '9780141441146', 'Charlotte Brontë', 'Smith, Elder & Co.', 1847, 'Gothic Fiction', 'Physical', 'B4', 3, 5, 5, NULL, NULL)"
]

# Execute all insertions
for query in sample_inserts:
    try:
        cursor.execute(query)
        print("✅ Inserted:", query.split("VALUES")[0].strip())
    except Exception as e:
        print("❌ Error:", e)

# Commit changes
connection.commit()
print("✅ All sample data inserted successfully.")

✅ Inserted: INSERT INTO LibraryBranches
✅ Inserted: INSERT INTO LibraryBranches
✅ Inserted: INSERT INTO LibraryBranches
✅ Inserted: INSERT INTO LibraryMembers
✅ Inserted: INSERT INTO LibraryMembers
✅ Inserted: INSERT INTO LibraryMembers
✅ Inserted: INSERT INTO LibraryMembers
✅ Inserted: INSERT INTO LibraryMembers
✅ Inserted: INSERT INTO Librarians
✅ Inserted: INSERT INTO Librarians
✅ Inserted: INSERT INTO Librarians
✅ Inserted: INSERT INTO Books
✅ Inserted: INSERT INTO Books
✅ Inserted: INSERT INTO Books
✅ Inserted: INSERT INTO Books
✅ Inserted: INSERT INTO Books
✅ Inserted: INSERT INTO Books
✅ Inserted: INSERT INTO Books
✅ Inserted: INSERT INTO Books
✅ Inserted: INSERT INTO Books
✅ Inserted: INSERT INTO Books
✅ All sample data inserted successfully.


**Populating loans and transactions table**

In [15]:
# Insert sample Loans
cursor.execute("""
INSERT INTO Loans (LoanID, BookID, MemberID, LoanDate, DueDate, Status)
VALUES (4001, 3001, 1001, SYSDATE - 10, SYSDATE - 3, 'Borrowed')
""")

cursor.execute("""
INSERT INTO Loans (LoanID, BookID, MemberID, LoanDate, DueDate, ReturnDate, Status)
VALUES (4002, 3002, 1002, SYSDATE - 20, SYSDATE - 5, SYSDATE, 'Returned')
""")

cursor.execute("""
INSERT INTO Loans (LoanID, BookID, MemberID, LoanDate, DueDate, Status)
VALUES (4003, 3004, 1003, SYSDATE - 7, SYSDATE + 7, 'Borrowed')
""")

# Insert sample Transactions
cursor.execute("""
INSERT INTO Transactions (TransactionID, MemberID, BookID, TransactionType, TransactionDate, Amount, Status, LibrarianID)
VALUES (5001, 1002, 3002, 'Fine Payment', SYSDATE, 5.00, 'Completed', 2002)
""")

cursor.execute("""
INSERT INTO Transactions (TransactionID, MemberID, BookID, TransactionType, TransactionDate, Amount, Status, LibrarianID)
VALUES (5002, 1003, 3003, 'Reservation', SYSDATE, 0, 'Pending', 2003)
""")

cursor.execute("""
INSERT INTO Transactions (TransactionID, MemberID, BookID, TransactionType, TransactionDate, Amount, Status, LibrarianID)
VALUES (5003, 1001, 3005, 'Reservation', SYSDATE, 0, 'Completed', 2001)
""")

# Commit changes
connection.commit()
print("✅ All sample Loans and Transactions inserted successfully.")

✅ All sample Loans and Transactions inserted successfully.


**Showing tables are actually populated**

In [4]:
def show_table(table_name):
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()
    columns = [col[0] for col in cursor.description]
    df = pd.DataFrame(rows, columns=columns)
    display(df)

In [41]:
show_table('librarymembers')

,MEMBERID,FIRSTNAME,LASTNAME,DATEOFBIRTH,EMAIL,PHONE,ADDRESS,MEMBERSHIPTYPE,MEMBERSHIPSTARTDATE,OUTSTANDINGFINES,CURRENTRESERVATIONS
0,1004,Linda,Martinez,1982-03-14,linda.martinez@example.com,555-4444,"101 Elm St, Metropolis",Faculty,2024-11-22 23:03:15,0.0,1
1,1001,Jessica,Smith,1985-07-15,james.smith@example.com,555-1111,"123 Maple St, Metropolis",Student,2023-03-22 18:51:28,0.0,5
2,1002,Maria,Garcia,1990-05-22,maria.garcia@example.com,555-2222,"456 Oak St, Metropolis",Student,2024-03-22 18:51:28,10.0,3
3,1003,Robert,Johnson,1978-11-02,robert.johnson@example.com,555-3333,"789 Pine St, Metropolis",Faculty,2022-03-22 18:51:28,5.0,2
4,1005,David,Brown,1995-09-30,david.brown@example.com,555-5555,"202 Birch St, Metropolis",Visitor,2024-09-22 18:51:28,2.0,4


In [8]:
show_table('transactions')

,TRANSACTIONID,MEMBERID,BOOKID,TRANSACTIONTYPE,TRANSACTIONDATE,AMOUNT,STATUS,LIBRARIANID
0,5001,1002,3002,Fine Payment,2025-03-22 22:32:48,5.0,Completed,2002
1,5002,1003,3003,Reservation,2025-03-22 22:32:48,0.0,Pending,2003
2,5003,1001,3005,Reservation,2025-03-22 22:32:48,0.0,Completed,2001


**Showing insertion works**

In [22]:
cursor.execute("SELECT * FROM LibraryMembers")
for row in cursor:
    print(row)

(1001, 'James', 'Smith', datetime.datetime(1985, 7, 15, 0, 0), 'james.smith@example.com', '555-1111', '123 Maple St, Metropolis', 'Student', datetime.datetime(2023, 3, 22, 18, 51, 28), 0.0, 5)
(1002, 'Maria', 'Garcia', datetime.datetime(1990, 5, 22, 0, 0), 'maria.garcia@example.com', '555-2222', '456 Oak St, Metropolis', 'Student', datetime.datetime(2024, 3, 22, 18, 51, 28), 10.0, 3)
(1003, 'Robert', 'Johnson', datetime.datetime(1978, 11, 2, 0, 0), 'robert.johnson@example.com', '555-3333', '789 Pine St, Metropolis', 'Faculty', datetime.datetime(2022, 3, 22, 18, 51, 28), 5.0, 2)
(1004, 'Linda', 'Martinez', datetime.datetime(1982, 3, 14, 0, 0), 'linda.martinez@example.com', '555-4444', '101 Elm St, Metropolis', 'Faculty', datetime.datetime(2024, 11, 22, 18, 51, 28), 0.0, 1)
(1005, 'David', 'Brown', datetime.datetime(1995, 9, 30, 0, 0), 'david.brown@example.com', '555-5555', '202 Birch St, Metropolis', 'Visitor', datetime.datetime(2024, 9, 22, 18, 51, 28), 2.0, 4)


**Showing how the trigger prevents an illegal entry**

In [23]:
cursor.execute(
"INSERT INTO LibraryMembers VALUES (1001, 'James', 'Smith', TO_DATE('2010-07-15', 'YYYY-MM-DD'), 'james.smith@example.com', '555-1111', '123 Maple St, Metropolis', 'Student', SYSDATE - INTERVAL '2' YEAR, 0, 5)",
)
connection.commit()

DatabaseError: ORA-20001: Member must be at least 18 years old.
ORA-06512: at "S25_KM242.TRG_CHECK_AGE", line 3
ORA-04088: error during execution of trigger 'S25_KM242.TRG_CHECK_AGE'

**Showing records can be updated**

In [25]:
cursor.execute("""
UPDATE LibraryMembers
SET FirstName = 'Jessica'
WHERE MemberID = 1001
""")
connection.commit()

In [26]:
cursor.execute("SELECT * FROM LibraryMembers")
for row in cursor:
    print(row)

(1001, 'Jessica', 'Smith', datetime.datetime(1985, 7, 15, 0, 0), 'james.smith@example.com', '555-1111', '123 Maple St, Metropolis', 'Student', datetime.datetime(2023, 3, 22, 18, 51, 28), 0.0, 5)
(1002, 'Maria', 'Garcia', datetime.datetime(1990, 5, 22, 0, 0), 'maria.garcia@example.com', '555-2222', '456 Oak St, Metropolis', 'Student', datetime.datetime(2024, 3, 22, 18, 51, 28), 10.0, 3)
(1003, 'Robert', 'Johnson', datetime.datetime(1978, 11, 2, 0, 0), 'robert.johnson@example.com', '555-3333', '789 Pine St, Metropolis', 'Faculty', datetime.datetime(2022, 3, 22, 18, 51, 28), 5.0, 2)
(1004, 'Linda', 'Martinez', datetime.datetime(1982, 3, 14, 0, 0), 'linda.martinez@example.com', '555-4444', '101 Elm St, Metropolis', 'Faculty', datetime.datetime(2024, 11, 22, 18, 51, 28), 0.0, 1)
(1005, 'David', 'Brown', datetime.datetime(1995, 9, 30, 0, 0), 'david.brown@example.com', '555-5555', '202 Birch St, Metropolis', 'Visitor', datetime.datetime(2024, 9, 22, 18, 51, 28), 2.0, 4)


**Deleting a record from the DB**

In [30]:
cursor.execute("""
DELETE FROM LibraryMembers
WHERE MemberID = 1004
""")
connection.commit()

In [31]:
cursor.execute("SELECT * FROM LibraryMembers")
for row in cursor:
    print(row)

(1001, 'Jessica', 'Smith', datetime.datetime(1985, 7, 15, 0, 0), 'james.smith@example.com', '555-1111', '123 Maple St, Metropolis', 'Student', datetime.datetime(2023, 3, 22, 18, 51, 28), 0.0, 5)
(1002, 'Maria', 'Garcia', datetime.datetime(1990, 5, 22, 0, 0), 'maria.garcia@example.com', '555-2222', '456 Oak St, Metropolis', 'Student', datetime.datetime(2024, 3, 22, 18, 51, 28), 10.0, 3)
(1003, 'Robert', 'Johnson', datetime.datetime(1978, 11, 2, 0, 0), 'robert.johnson@example.com', '555-3333', '789 Pine St, Metropolis', 'Faculty', datetime.datetime(2022, 3, 22, 18, 51, 28), 5.0, 2)
(1005, 'David', 'Brown', datetime.datetime(1995, 9, 30, 0, 0), 'david.brown@example.com', '555-5555', '202 Birch St, Metropolis', 'Visitor', datetime.datetime(2024, 9, 22, 18, 51, 28), 2.0, 4)


In [32]:
cursor.execute(
"INSERT INTO LibraryMembers VALUES (1004, 'Linda', 'Martinez', TO_DATE('1982-03-14', 'YYYY-MM-DD'), 'linda.martinez@example.com', '555-4444', '101 Elm St, Metropolis', 'Faculty', SYSDATE - INTERVAL '4' MONTH, 0, 1)"
)
connection.commit()

**Function to show results of functions**

In [34]:
def show_view(view_name):
    cursor.execute(f"SELECT * FROM {view_name}")
    rows = cursor.fetchall()
    columns = [col[0] for col in cursor.description]
    df = pd.DataFrame(rows, columns=columns)
    display(df)

In [35]:
show_view("View_LibrarianTransactions")

,TRANSACTIONID,MEMBER,TITLE,TRANSACTIONTYPE,AMOUNT,TRANSACTIONDATE,STATUS
0,5001,Maria Garcia,1984,Fine Payment,5.0,2025-03-22 22:32:48,Completed
1,5002,Robert Johnson,Pride and Prejudice,Reservation,0.0,2025-03-22 22:32:48,Pending
2,5003,Jessica Smith,Moby-Dick,Reservation,0.0,2025-03-22 22:32:48,Completed


In [36]:
show_view("View_AvailableBooks")

,BOOKID,TITLE,AUTHOR,GENRE,BOOKTYPE,AVAILABLECOPIES,BRANCHID
0,3001,To Kill a Mockingbird,Harper Lee,Fiction,Physical,4,1
1,3002,1984,George Orwell,Dystopian,Physical,3,1
2,3003,Pride and Prejudice,Jane Austen,Romance,Physical,3,1
3,3004,The Great Gatsby,F. Scott Fitzgerald,Tragedy,Physical,5,1
4,3005,Moby-Dick,Herman Melville,Adventure,Physical,2,2
5,3006,Brave New World,Aldous Huxley,Science Fiction,Ebook,2,2
6,3007,The Catcher in the Rye,J.D. Salinger,Coming-of-Age,Physical,4,2
7,3008,The Hobbit,J.R.R. Tolkien,Fantasy,Physical,3,3
8,3009,Crime and Punishment,Fyodor Dostoevsky,Philosophical Fiction,Ebook,1,3
9,3010,Jane Eyre,Charlotte Brontë,Gothic Fiction,Physical,5,3


**Showing results of some queries**

In [43]:
def run_query(query):
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [col[0] for col in cursor.description]
    df = pd.DataFrame(rows, columns=columns)
    display(df)

In [45]:
run_query("""
SELECT L.LoanID,
       M.FirstName || ' ' || M.LastName AS MemberName,
       B.Title AS BookTitle,
       L.LoanDate,
       L.DueDate,
       L.Status
FROM Loans L
JOIN LibraryMembers M ON L.MemberID = M.MemberID
JOIN Books B ON L.BookID = B.BookID
WHERE L.Status = 'Borrowed'
""")

,LOANID,MEMBERNAME,BOOKTITLE,LOANDATE,DUEDATE,STATUS
0,4001,Jessica Smith,To Kill a Mockingbird,2025-03-12 22:32:48,2025-03-19 22:32:48,Borrowed
1,4003,Robert Johnson,The Great Gatsby,2025-03-15 22:32:48,2025-03-29 22:32:48,Borrowed


In [46]:
run_query("""
SELECT T.TransactionID,
       M.FirstName || ' ' || M.LastName AS Member,
       B.Title AS Book,
       T.TransactionDate,
       T.Status
FROM Transactions T
JOIN LibraryMembers M ON T.MemberID = M.MemberID
JOIN Books B ON T.BookID = B.BookID
WHERE T.TransactionType = 'Reservation'
""")

,TRANSACTIONID,MEMBER,BOOK,TRANSACTIONDATE,STATUS
0,5002,Robert Johnson,Pride and Prejudice,2025-03-22 22:32:48,Pending
1,5003,Jessica Smith,Moby-Dick,2025-03-22 22:32:48,Completed
